In [1]:
import os
import pandas as pd
from scrapers import *
from keras.models import load_model
from sklearn.preprocessing import StandardScaler

DATA_PATH = 'data/'

Using TensorFlow backend.
/home/stephen/Projects/betting_model/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stephen/Projects/betting_model/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stephen/Projects/betting_model/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h

# Predict upcoming games

To predict upcoming games we will need to:
1. Scrape upcoming fixtures with odds
2. Concatentate these onto results so far for season 19/20
3. Use the same functions previously to create data for our model

# Get EMA stats

The most tricky part is getting EMA stats for the upcoming fixtures, the league stats should be straight forward. 

In [2]:
scraper = FixtureScraper()
fixtures = scraper.get_fixtures()
scraper.close_driver()

fixtures.replace({'Man Utd': 'Man United',
                  'C Palace': 'Crystal Palace',
                  'Sheff Utd': 'Sheffield United',
                  'Sheffield Utd': 'Sheffield United'},
                   inplace=True)

In [3]:
df_ema = (pd.read_csv(os.path.join(DATA_PATH, 'all_seasons_joined.csv'))
                    .assign(Date=lambda df: pd.to_datetime(df.Date))
                    .pipe(lambda df: df.dropna(thresh=len(df) - 2, axis=1))  # Drop cols with NAs
                    .dropna(axis=0)  # Drop rows with NAs
                    .sort_values('season')
                    .append(fixtures, sort=True)
                    .reset_index(drop=True)
                    .assign(gameId=lambda df: list(df.index + 1))
                    )

/home/stephen/Projects/betting_model/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (80) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_ema.tail()

,AC,AF,AR,AS,AST,AY,AwayTeam,B365A,B365D,B365H,...,HY,HomeTeam,Referee,Unnamed: 0,Unnamed: 0.1,VCA,VCD,VCH,season,gameId
4694,NaN,NaN,NaN,NaN,NaN,NaN,Norwich,8.50,5.25,1.33,...,NaN,Everton,NaN,NaN,NaN,NaN,NaN,NaN,1920,4695
4695,NaN,NaN,NaN,NaN,NaN,NaN,Burnley,3.40,3.25,2.20,...,NaN,Watford,NaN,NaN,NaN,NaN,NaN,NaN,1920,4696
4696,NaN,NaN,NaN,NaN,NaN,NaN,Chelsea,7.00,5.50,1.36,...,NaN,Man City,NaN,NaN,NaN,NaN,NaN,NaN,1920,4697
4697,NaN,NaN,NaN,NaN,NaN,NaN,Man United,2.05,3.40,3.75,...,NaN,Sheffield United,NaN,NaN,NaN,NaN,NaN,NaN,1920,4698
4698,NaN,NaN,NaN,NaN,NaN,NaN,Newcastle,3.30,3.40,2.20,...,NaN,Aston Villa,NaN,NaN,NaN,NaN,NaN,NaN,1920,4699


In [5]:
# Drop betting odds as we can use league stats function to get these
df_ema.drop(['B365H', 'B365D', 'B365A'], 1, inplace=True)

In [6]:
# Define a function which restructures our DataFrame
def create_multiline_df_stats(old_stats_df):
    # Create a list of columns we want and their mappings to more interpretable names
    home_stats_cols = ['Date', 'season', 'HomeTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
                       'HR', 'AR']
    
    away_stats_cols = ['Date', 'season', 'AwayTeam', 'FTAG', 'FTHG', 'HTAG', 'HTHG', 'AS', 'HS', 'AST', 'HST', 'AF', 'HF', 'AC', 'HC', 'AY', 'HY',
                       'AR', 'HR']
    
    stats_cols_mapping = ['Date', 'season', 'Team', 'goalsFor', 'goalsAgainst', 'halfTimeGoalsFor', 
                          'halfTimeGoalsAgainst', 'shotsFor', 'shotsAgainst', 'shotsOnTargetFor',
                          'shotsOnTargetAgainst', 'freesFor', 'freesAgainst', 'cornersFor', 
                          'cornersAgainst', 'yellowsFor', 'yellowsAgainst', 'redsFor', 'redsAgainst']
    
    # Create a dictionary of the old column names to new column names
    home_mapping = {old_col: new_col for old_col, new_col in zip(home_stats_cols, stats_cols_mapping)}
    away_mapping = {old_col: new_col for old_col, new_col in zip(away_stats_cols, stats_cols_mapping)}
    
    # Put each team onto an individual row
    multi_line_stats = (old_stats_df[['gameId'] + home_stats_cols] # Filter for only the home team columns
                    .rename(columns=home_mapping) # Rename the columns
                    .assign(homeGame=1) # Assign homeGame=1 so that we can use a general function later
                    .append((old_stats_df[['gameId'] + away_stats_cols]) # Append the away team columns
                            .rename(columns=away_mapping) # Rename the away team columns
                            .assign(homeGame=0), sort=True)
                    .sort_values(by='gameId') # Sort the values
                    .reset_index(drop=True))
    return multi_line_stats

In [7]:
# Define a function which creates an EMA DataFrame from the stats DataFrame
def create_stats_features_ema(stats, span):
    # Create a restructured DataFrames so that we can calculate EMA
    multi_line_stats = create_multiline_df_stats(stats)
    
    # Create a copy of the DataFrame
    ema_features = multi_line_stats[['Date', 'season', 'gameId', 'Team', 'homeGame']].copy()
    
    # Get the columns that we want to create EMA for
    feature_names = multi_line_stats.drop(columns=['Date', 'season', 'gameId', 'Team', 'homeGame']).columns
    
    # Loop over the features
    for feature_name in feature_names:
        feature_ema = (multi_line_stats.groupby('Team')[feature_name] # Calculate the EMA
                                                  .transform(lambda row: row.ewm(span=span, min_periods=2)
                                                             .mean()
                                                             .shift(1))) # Shift the data down 1 so we don't leak data
        ema_features[feature_name] = feature_ema # Add the new feature to the DataFrame
    return ema_features

In [8]:
df_ema = create_stats_features_ema(df_ema, 50)
df_ema.tail()

,Date,season,gameId,Team,homeGame,cornersAgainst,cornersFor,freesAgainst,freesFor,goalsAgainst,...,halfTimeGoalsAgainst,halfTimeGoalsFor,redsAgainst,redsFor,shotsAgainst,shotsFor,shotsOnTargetAgainst,shotsOnTargetFor,yellowsAgainst,yellowsFor
9393,23/11/19,1920,4697,Man City,1,2.564720,8.106118,8.305311,9.128231,0.876450,...,0.500225,1.348264,0.026781,0.053185,6.918444,19.025578,2.860819,6.825688,1.447244,1.457315
9394,23/11/19,1920,4698,Man United,0,4.410122,5.537868,12.190229,11.262102,1.270501,...,0.483735,0.870143,0.058399,0.051718,11.742318,13.822989,4.381545,5.504194,2.062852,1.977780
9395,23/11/19,1920,4698,Sheffield United,1,5.914704,6.095539,8.108339,10.843066,0.980354,...,0.331755,0.442165,0.000000,0.092912,11.587289,10.430143,3.734383,3.424489,1.303453,1.698775
9396,23/11/19,1920,4699,Aston Villa,1,6.593091,4.434333,11.505720,11.408889,1.802244,...,0.599397,0.457437,0.133342,0.078842,15.120976,11.275743,4.786292,4.122362,1.833025,1.808538
9397,23/11/19,1920,4699,Newcastle,0,6.206665,4.325239,9.594730,10.256995,1.464552,...,0.583970,0.555547,0.006138,0.096214,13.665840,11.091432,4.450516,3.613229,1.582571,1.574617


In [9]:
def restructure_stats_features(stats_features):
    non_features = ['homeGame', 'Team', 'gameId']

    stats_features_restructured = (stats_features.query('homeGame == 1')
                                    .rename(columns={col: 'f_' + col + 'Home' for col in stats_features.columns if col not in non_features})
                                    .rename(columns={'Team': 'HomeTeam'})
                                    .pipe(pd.merge, (stats_features.query('homeGame == 0')
                                                        .rename(columns={'Team': 'AwayTeam'})
                                                        .rename(columns={col: 'f_' + col + 'Away' for col in stats_features.columns 
                                                                         if col not in non_features})), on=['gameId'])
                                    .dropna())
    return stats_features_restructured

df_ema = restructure_stats_features(df_ema)
df_ema.tail()

,f_DateHome,f_seasonHome,gameId,HomeTeam,homeGame_x,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,...,f_halfTimeGoalsAgainstAway,f_halfTimeGoalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway,f_yellowsAgainstAway,f_yellowsForAway
4694,23/11/19,1920,4695,Everton,1,4.648396,5.929136,11.001541,11.661006,1.417041,...,0.915155,0.418876,0.049579,0.047436,15.056477,10.967077,5.430868,3.613490,1.356096,1.670512
4695,23/11/19,1920,4696,Watford,1,5.523914,5.067102,9.307101,11.230567,1.784747,...,0.751969,0.555748,0.075602,0.007623,15.996351,9.906070,5.091335,3.381323,1.102123,1.847012
4696,23/11/19,1920,4697,Man City,1,2.564720,8.106118,8.305311,9.128231,0.876450,...,0.471474,0.928125,0.005025,0.011362,9.184563,16.264718,3.114963,5.712200,1.837185,1.582328
4697,23/11/19,1920,4698,Sheffield United,1,5.914704,6.095539,8.108339,10.843066,0.980354,...,0.483735,0.870143,0.058399,0.051718,11.742318,13.822989,4.381545,5.504194,2.062852,1.977780
4698,23/11/19,1920,4699,Aston Villa,1,6.593091,4.434333,11.505720,11.408889,1.802244,...,0.583970,0.555547,0.006138,0.096214,13.665840,11.091432,4.450516,3.613229,1.582571,1.574617


In [10]:
df_ema.drop(['f_DateHome', 'f_seasonHome', 'HomeTeam',
               'homeGame_x', 'f_yellowsAgainstAway', 'f_yellowsForAway',
               'f_halfTimeGoalsAgainstHome', 'f_halfTimeGoalsForHome','f_halfTimeGoalsAgainstAway', 'f_halfTimeGoalsForAway',
               'f_yellowsAgainstHome', 'f_yellowsForHome', 'f_DateAway', 'f_seasonAway', 
               'AwayTeam', 'homeGame_y'], 1, inplace=True)

In [11]:
df_ema.columns

Index(['gameId', 'f_cornersAgainstHome', 'f_cornersForHome',
       'f_freesAgainstHome', 'f_freesForHome', 'f_goalsAgainstHome',
       'f_goalsForHome', 'f_redsAgainstHome', 'f_redsForHome',
       'f_shotsAgainstHome', 'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway'],
      dtype='object')

# Get League stats

To get the league stats we will use the same 'features' DataFrame that we used to create EMA stats. We can use the same function we used previously to create league data.

In [12]:
def get_points(goals_scored, goals_conceded):
    if goals_scored > goals_conceded: 
        points = 3
    elif goals_conceded == goals_scored:
        points = 1
    else:
        points = 9
    return points

def get_result(goals_scored, goals_conceded):
    if goals_scored > goals_conceded: 
        result = 'W'
    elif goals_conceded == goals_scored:
        result = 'D'
    else:
        result = 'L'
    return result


def get_last_result(team):
    """
    Helper function which takes a team-name as a string and returns goal stats from
    their most recent match
    """
    with open(DATA_PATH + 'season1920.csv') as f:
        for row in reversed(list(csv.reader(f))):
            if team == row[4]:
                goals_scored = int(row[6])
                goals_conceded = int(row[7])

                return(goals_scored, goals_conceded)
            elif team == row[5]:
                goals_scored = int(row[7])
                goals_conceded = int(row[6])

                return goals_scored, goals_conceded

In [13]:
def create_row(home_team, away_team, home_odds, draw_odds, away_odds, matchweek):
    # First get last result data
    home_goals_scored, home_goals_against = get_last_result(home_team)
    home_result = get_result(home_goals_scored, home_goals_against)
    home_points = get_points(home_goals_scored, home_goals_against)
    
    away_goals_scored, away_goals_against = get_last_result(away_team)
    away_result = get_result(away_goals_scored, away_goals_against)
    away_points = get_points(away_goals_scored, away_goals_against)
    
    # Combine this with team data we already have, I created a csv
    # using existing fucntions without scaling.   
    with open(os.path.join(DATA_PATH, 'season1920_data.csv')) as f:
        reader = csv.reader(f)
        next(reader, None)
        for row in reader:
            if home_team == row[3]:
                htp = (int(row[12]) + home_points) / matchweek
                hm1 = home_result
                hm2 = row[17]
                hm3 = row[18]
                htgd = (float(row[35]) + home_goals_scored - home_goals_against) / matchweek
                ht_prev_pos = int(row[28])
                ht_form_pts = int(row[33])
                
            elif away_team == row[3]:
                atp = (int(row[12]) + away_points) / matchweek
                am1 = away_result
                am2 = row[17]
                am3 = row[18]
                atgd = (float(row[35]) + away_goals_scored - away_goals_against) / matchweek
                at_prev_pos = int(row[28])
                at_form_pts = int(row[33])
                
            elif home_team == row[4]:
                htp = (int(row[13]) + home_points) / matchweek
                hm1 = home_result
                hm2 = row[22]
                hm3 = row[23]
                htgd = (float(row[36]) + home_goals_scored - home_goals_against) / matchweek
                ht_prev_pos = int(row[29])
                ht_form_pts = int(row[34])
            elif away_team == row[4]:
                atp = (int(row[13]) + away_points) / matchweek
                am1 = away_result
                am2 = row[22]
                am3 = row[23]
                atgd = (float(row[36]) + away_goals_scored - away_goals_against) / matchweek
                at_prev_pos = int(row[29])
                at_form_pts = int(row[34])

        diff_lp = ht_prev_pos - at_prev_pos
        diff_points = (atp - htp) / matchweek
        diff_form_points = (ht_form_pts - at_form_pts) / matchweek
          
        row = [htp, atp, home_odds, draw_odds, away_odds, 
               hm1, hm2, hm3, am1, am2, am3, htgd, atgd, 
               diff_points, diff_form_points, diff_lp]
        
        # print(row)
        return row

In [14]:
headers = ['HTP', 'ATP', 'B365H', 'B365D', 'B365A', 
           'HM1', 'HM2', 'HM3', 'AM1', 'AM2', 'AM3', 
           'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP']

with open(os.path.join(DATA_PATH, 'fixtures_league_data.csv'), 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    for i, row in fixtures.iterrows():
        # Last item in row is MatchWeek, entering manually for now
        row = create_row(row.HomeTeam, row.AwayTeam, row.B365H, 
                         row.B365D, row.B365A, 13.0)
        writer.writerow(row)

In [15]:
df_ema.columns

Index(['gameId', 'f_cornersAgainstHome', 'f_cornersForHome',
       'f_freesAgainstHome', 'f_freesForHome', 'f_goalsAgainstHome',
       'f_goalsForHome', 'f_redsAgainstHome', 'f_redsForHome',
       'f_shotsAgainstHome', 'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway'],
      dtype='object')

# Combine datasets


In [16]:
# Load league data from csv file
league_data = pd.read_csv('data/fixtures_league_data.csv')
league_data.drop(['HM1', 'HM2', 'HM3', 
                  'AM1', 'AM2', 'AM3'], 1, inplace=True)

In [17]:
league_data.columns

Index(['HTP', 'ATP', 'B365H', 'B365D', 'B365A', 'HTGD', 'ATGD', 'DiffPts',
       'DiffFormPts', 'DiffLP'],
      dtype='object')

In [18]:
# We only need upcoming fixtures from EMA stats
df_ema1920 = df_ema.tail(len(fixtures)).reset_index()

In [19]:
df = pd.concat([df_ema1920, league_data], axis=1)

In [20]:
df.drop(['index', 'gameId', ], 1, inplace=True)

In [21]:
df_ema1920

,index,gameId,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,f_goalsForHome,f_redsAgainstHome,f_redsForHome,...,f_freesAgainstAway,f_freesForAway,f_goalsAgainstAway,f_goalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway
0,4689,4690,5.318883,5.001668,11.048701,9.603531,1.614435,1.325661,0.012410,0.063928,...,10.847425,10.555322,1.196352,1.851004,0.025776,0.105628,13.087531,13.661734,4.791719,4.945787
1,4690,4691,5.339666,6.572225,12.355065,10.699987,1.394766,1.751072,0.046836,0.057889,...,9.661503,11.047751,1.933536,1.103309,0.112058,0.098739,14.031607,12.733435,4.939587,4.296904
2,4691,4692,6.182648,5.268830,11.759702,8.855945,1.619882,1.359672,0.118264,0.048312,...,8.974618,11.336053,1.381412,1.239985,0.121409,0.094060,12.943410,12.067226,4.666737,4.246228
3,4692,4693,5.537214,4.038127,8.717919,11.862341,1.501477,1.010347,0.052657,0.134104,...,10.428963,9.888379,0.979950,1.703437,0.143695,0.087676,10.934460,13.084528,3.474635,4.631751
4,4693,4694,5.749503,5.142353,11.766969,11.181507,1.387541,1.053678,0.146282,0.020070,...,9.043299,8.887131,0.704562,2.339699,0.015046,0.034324,8.924656,16.037339,2.664796,6.149405
5,4694,4695,4.648396,5.929136,11.001541,11.661006,1.417041,1.264413,0.113304,0.111293,...,9.781712,10.472629,1.859702,1.003284,0.049579,0.047436,15.056477,10.967077,5.430868,3.613490
6,4695,4696,5.523914,5.067102,9.307101,11.230567,1.784747,1.047818,0.054073,0.102449,...,10.598268,9.992710,1.624213,1.220076,0.075602,0.007623,15.996351,9.906070,5.091335,3.381323
7,4696,4697,2.564720,8.106118,8.305311,9.128231,0.876450,2.743550,0.026781,0.053185,...,11.510563,9.004256,1.164201,1.909446,0.005025,0.011362,9.184563,16.264718,3.114963,5.712200
8,4697,4698,5.914704,6.095539,8.108339,10.843066,0.980354,1.247720,0.000000,0.092912,...,12.190229,11.262102,1.270501,1.647939,0.058399,0.051718,11.742318,13.822989,4.381545,5.504194
9,4698,4699,6.593091,4.434333,11.505720,11.408889,1.802244,1.062181,0.133342,0.078842,...,9.594730,10.256995,1.464552,1.053008,0.006138,0.096214,13.665840,11.091432,4.450516,3.613229


# Make Predictions

Now that we have prepared the data we can load our model and make predictions.

In [22]:
scaler = StandardScaler()
X = scaler.fit_transform(df)

In [23]:
model = load_model('25Nov19.h5')

In [24]:
predictions = model.predict(X)

In [25]:
predictions

array([[0.33883515, 0.30644625, 0.35471863],
       [0.95096827, 0.04170317, 0.00732855],
       [0.5765048 , 0.22627163, 0.19722351],
       [0.00162994, 0.05776063, 0.94060946],
       [0.0032174 , 0.07534139, 0.9214412 ],
       [0.84266114, 0.13497005, 0.02236881],
       [0.2648822 , 0.34616342, 0.38895434],
       [0.93662935, 0.05270518, 0.01066548],
       [0.8009505 , 0.14810276, 0.0509467 ],
       [0.8226241 , 0.14137332, 0.03600264]], dtype=float32)

In [26]:
for i in range(len(X)):
    pred = np.argmax(predictions[i])
    if pred == 0:
        odds_diff = predictions[i][pred] - (1/league_data['B365H'][i])
        print(i, 'Prediction:', pred, 'Odds diff:', odds_diff)
    elif pred == 1:
        odds_diff = predictions[i][pred] - (1/league_data['B365D'][i])
        print(i, 'Prediction:', pred, 'Odds diff:', odds_diff)
    else:
        odds_diff = predictions[i][pred] - (1/league_data['B365A'][i])
        print(i, 'Prediction:', pred, 'Odds diff:', odds_diff)

0 Prediction: 2 Odds diff: -0.2476910130087152
1 Prediction: 0 Odds diff: 0.2613130931196541
2 Prediction: 0 Odds diff: 0.22807277079658644
3 Prediction: 2 Odds diff: 0.4277889422881297
4 Prediction: 2 Odds diff: 0.2547745307286581
5 Prediction: 0 Odds diff: 0.0907814431011229
6 Prediction: 2 Odds diff: 0.09483669435276704
7 Prediction: 0 Odds diff: 0.2013352373067071
8 Prediction: 0 Odds diff: 0.5342838605244955
9 Prediction: 0 Odds diff: 0.36807863278822467


# Sanity check

To try to make sure we have correct data we will make predictions on matches this season that have already happenend and then compare these do our prediction we made previously at ~64%.

To do this we need to first combine EMA_stats with league stats, similar to above process.

In [27]:
league_data = pd.read_csv('data/league_data.csv')
league_data.drop([ 'Date', 'HomeTeam', 'AwayTeam', 
                  'gameId', 'HTGS', 'ATGS', 'HTGC',
                  'ATGC', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 
                  'AM2', 'AM3', 'AM4', 'AM5', 'HTFormPts',
                 'ATFormPts', 'HTFormPtsStr', 'ATFormPtsStr'], 1, inplace=True)

In [28]:
league_data = league_data.loc[league_data['season'] == 1920].reset_index(drop=True)
league_data.drop([10, 30], inplace=True)
league_data.reset_index(drop=True)

,Unnamed: 0,season,FTR,HTP,ATP,B365H,B365D,B365A,MW,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,4560,1920,H,0.000000,0.000000,1.14,10.00,19.00,1.0,0.000000,0.000000,0.000000,0.000000,0
1,4561,1920,A,0.000000,0.000000,12.00,6.50,1.22,1.0,0.000000,0.000000,0.000000,0.000000,0
2,4562,1920,D,0.000000,0.000000,1.95,3.60,3.60,1.0,0.000000,0.000000,0.000000,0.000000,0
3,4563,1920,H,0.000000,0.000000,2.62,3.20,2.75,1.0,0.000000,0.000000,0.000000,0.000000,0
4,4564,1920,D,0.000000,0.000000,3.00,3.25,2.37,1.0,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,4685,1920,A,1.076923,0.538462,1.36,5.10,8.00,13.0,-0.538462,-0.923077,0.538462,0.461538,0
124,4686,1920,A,0.615385,1.153846,2.30,3.40,3.10,13.0,-1.153846,-0.076923,-0.538462,0.000000,0
125,4687,1920,H,1.923077,2.000000,1.44,5.00,6.50,13.0,1.384615,1.000000,-0.076923,-0.461538,0
126,4688,1920,D,1.307692,1.230769,3.50,3.40,2.10,13.0,0.307692,0.000000,0.076923,0.153846,0


In [29]:
league_data

,Unnamed: 0,season,FTR,HTP,ATP,B365H,B365D,B365A,MW,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,4560,1920,H,0.000000,0.000000,1.14,10.00,19.00,1.0,0.000000,0.000000,0.000000,0.000000,0
1,4561,1920,A,0.000000,0.000000,12.00,6.50,1.22,1.0,0.000000,0.000000,0.000000,0.000000,0
2,4562,1920,D,0.000000,0.000000,1.95,3.60,3.60,1.0,0.000000,0.000000,0.000000,0.000000,0
3,4563,1920,H,0.000000,0.000000,2.62,3.20,2.75,1.0,0.000000,0.000000,0.000000,0.000000,0
4,4564,1920,D,0.000000,0.000000,3.00,3.25,2.37,1.0,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,4685,1920,A,1.076923,0.538462,1.36,5.10,8.00,13.0,-0.538462,-0.923077,0.538462,0.461538,0
126,4686,1920,A,0.615385,1.153846,2.30,3.40,3.10,13.0,-1.153846,-0.076923,-0.538462,0.000000,0
127,4687,1920,H,1.923077,2.000000,1.44,5.00,6.50,13.0,1.384615,1.000000,-0.076923,-0.461538,0
128,4688,1920,D,1.307692,1.230769,3.50,3.40,2.10,13.0,0.307692,0.000000,0.076923,0.153846,0


In [30]:
league_data.columns

Index(['Unnamed: 0', 'season', 'FTR', 'HTP', 'ATP', 'B365H', 'B365D', 'B365A',
       'MW', 'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP'],
      dtype='object')

In [31]:
league_data.drop(['MW', 'Unnamed: 0', 'season'], 1, inplace=True)
# league_data.reset_index(inplace=True)

In [32]:
league_data.head()

,FTR,HTP,ATP,B365H,B365D,B365A,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,H,0.0,0.0,1.14,10.00,19.00,0.0,0.0,0.0,0.0,0
1,A,0.0,0.0,12.00,6.50,1.22,0.0,0.0,0.0,0.0,0
2,D,0.0,0.0,1.95,3.60,3.60,0.0,0.0,0.0,0.0,0
3,H,0.0,0.0,2.62,3.20,2.75,0.0,0.0,0.0,0.0,0
4,D,0.0,0.0,3.00,3.25,2.37,0.0,0.0,0.0,0.0,0


In [33]:
len(league_data.columns)

11

In [34]:
df_ema.columns

Index(['gameId', 'f_cornersAgainstHome', 'f_cornersForHome',
       'f_freesAgainstHome', 'f_freesForHome', 'f_goalsAgainstHome',
       'f_goalsForHome', 'f_redsAgainstHome', 'f_redsForHome',
       'f_shotsAgainstHome', 'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway'],
      dtype='object')

In [35]:
len(df_ema)

4642

In [36]:
ema1920 = df_ema.loc[(df_ema['gameId'] > 4559) & (df_ema['gameId'] < 4680)].reset_index()

In [37]:
len(ema1920)

118

In [38]:
len(league_data)

128

In [39]:
df1920 = pd.concat([ema1920, league_data], axis=1)

In [40]:
df1920.columns

Index(['index', 'gameId', 'f_cornersAgainstHome', 'f_cornersForHome',
       'f_freesAgainstHome', 'f_freesForHome', 'f_goalsAgainstHome',
       'f_goalsForHome', 'f_redsAgainstHome', 'f_redsForHome',
       'f_shotsAgainstHome', 'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway', 'FTR', 'HTP', 'ATP', 'B365H', 'B365D',
       'B365A', 'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP'],
      dtype='object')

In [41]:
df1920.drop(['index', 'gameId'], 1, inplace=True)

In [42]:
df1920.columns

Index(['f_cornersAgainstHome', 'f_cornersForHome', 'f_freesAgainstHome',
       'f_freesForHome', 'f_goalsAgainstHome', 'f_goalsForHome',
       'f_redsAgainstHome', 'f_redsForHome', 'f_shotsAgainstHome',
       'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway', 'FTR', 'HTP', 'ATP', 'B365H', 'B365D',
       'B365A', 'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP'],
      dtype='object')

In [43]:
X = df1920.drop(['FTR'], 1)
y_true = df1920['FTR']

In [44]:
X.shape[1]

34

In [45]:
def transform_results(results):
    transformed = []
    for i in range(len(results)):
        if results[i] == 'H':
            transformed.append(0)
        elif results[i] == 'A':
            transformed.append(2)
        else:
            transformed.append(1)
    return np.array(transformed)

y_true = transform_results(y_true)

In [46]:
y_true

array([0, 2, 1, 0, 1, 2, 0, 1, 2, 0, 1, 2, 1, 0, 0, 2, 1, 0, 1, 1, 0, 2,
       2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 0, 1, 0, 2,
       0, 2, 0, 0, 1, 1, 2, 0, 0, 2, 0, 1, 1, 0, 0, 2, 2, 1, 1, 0, 0, 0,
       0, 2, 0, 1, 0, 0, 0, 2, 1, 2, 0, 2, 2, 0, 0, 0, 1, 0, 0, 1, 1, 2,
       1, 0, 2, 0, 0, 1, 1, 2, 1, 1, 0, 2, 0, 1, 2, 0, 0, 0, 2, 2, 2, 1,
       2, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 2, 2, 2, 2, 2, 0, 1, 1])

In [47]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [48]:
model = load_model('17Nov19.h5')


_, acc = model.evaluate(X, y_true)
acc

130/130 [==============================] - 0s 320us/step


0.5769230723381042

In [49]:
y_preds = model.predict(X)

funds = 100
wager = 10
favourites = 0
no_bets = 0
min_diff = 0.03

for i in range(len(X)):
    prediction = np.argmax(y_preds[i])
    print('\nPrediction', prediction)
    print('Actual', y_true[i])
    print('Favourite', np.argmin([df1920['B365H'][i], df1920['B365D'][i], 
                                  df1920['B365A'][i]]))
    print('Prediction proba', y_preds[i])
    print('Home, Draw and Away odds', df1920['B365H'][i],       
          df1920['B365D'][i], df1920['B365A'][i])
    
        

    if prediction == 0:
        odds_diff = y_preds[i][prediction] - (1/df1920['B365H'][i])
        # If odds_diff positive place bet
        if odds_diff > min_diff:
            if prediction == np.argmin([df1920['B365H'][i], df1920['B365D'][i], 
                                  df1920['B365A'][i]]):
                favourites +=1
                
            if  prediction == y_true[i]:
                funds += (wager * df1920['B365H'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
    elif prediction == 1:
        odds_diff = y_preds[i][prediction] - (1/df1920['B365D'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([df1920['B365H'][i], df1920['B365D'][i], 
                                  df1920['B365A'][i]]):
                favourites +=1
            if  prediction == y_true[i]:
                funds +=( wager * df1920['B365D'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
    else:
        odds_diff = y_preds[i][prediction] - (1/df1920['B365A'][i])
        if odds_diff >  min_diff:
            if prediction == np.argmin([df1920['B365H'][i], df1920['B365D'][i], 
                                  df1920['B365A'][i]]):
                favourites +=1
            if  prediction == y_true[i]:
                funds += (wager * df1920['B365A'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1

        
    
    
    print('Funds', funds)
    
print(f'Betted on favourite {favourites} times out of {len(X)} matches.')
print(f'No bet placed {no_bets} times')
         


Prediction 0
Actual 0
Favourite 0
Prediction proba [0.5801569  0.2492719  0.17057115]
Home, Draw and Away odds 1.14 10.0 19.0
Funds 100

Prediction 2
Actual 2
Favourite 2
Prediction proba [0.28566477 0.27336773 0.44096753]
Home, Draw and Away odds 12.0 6.5 1.22
Funds 100

Prediction 0
Actual 1
Favourite 0
Prediction proba [0.39058346 0.3023451  0.3070714 ]
Home, Draw and Away odds 1.95 3.6 3.6
Funds 100

Prediction 1
Actual 0
Favourite 0
Prediction proba [0.28829902 0.37467125 0.33702976]
Home, Draw and Away odds 2.62 3.2 2.75
Funds 90

Prediction 2
Actual 1
Favourite 2
Prediction proba [0.31983787 0.31477454 0.3653876 ]
Home, Draw and Away odds 3.0 3.25 2.37
Funds 90

Prediction 0
Actual 2
Favourite 0
Prediction proba [0.35912284 0.35103735 0.28983983]
Home, Draw and Away odds 1.9 3.4 4.0
Funds 90

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.3952686  0.28187263 0.3228588 ]
Home, Draw and Away odds 1.3 5.25 10.0
Funds 90

Prediction 2
Actual 1
Favourite 0
Prediction proba [0

Prediction proba [0.51503617 0.3000656  0.18489824]
Home, Draw and Away odds 1.45 4.75 6.5
Funds 804.9000000000002

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.7416981  0.17680036 0.08150156]
Home, Draw and Away odds 1.5 4.33 6.5
Funds 809.9000000000002

Prediction 2
Actual 2
Favourite 2
Prediction proba [0.1281475  0.29678163 0.57507086]
Home, Draw and Away odds 4.33 4.0 1.75
Funds 809.9000000000002

Prediction 0
Actual 0
Favourite 2
Prediction proba [0.6345956  0.28307435 0.08233006]
Home, Draw and Away odds 3.25 3.6 2.15
Funds 832.4000000000002

Prediction 1
Actual 1
Favourite 0
Prediction proba [0.310491   0.41042197 0.279087  ]
Home, Draw and Away odds 1.75 3.75 4.75
Funds 859.9000000000002

Prediction 0
Actual 2
Favourite 2
Prediction proba [0.5216961  0.25847855 0.21982536]
Home, Draw and Away odds 9.0 5.0 1.33
Funds 849.9000000000002

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.5230571  0.31706712 0.15987583]
Home, Draw and Away odds 1.72 4.2 4.33
Funds 849.

In [50]:
headers = ['f_cornersAgainstHome', 'f_cornersForHome', 'f_freesAgainstHome',
       'f_freesForHome', 'f_goalsAgainstHome', 'f_goalsForHome',
       'f_redsAgainstHome', 'f_redsForHome', 'f_shotsAgainstHome',
       'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway', 'HTP', 'ATP', 'B365H', 'B365D',
       'B365A', 'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP']
X_testdf = pd.DataFrame(np.load('X_test.npy'), columns=headers)

In [51]:
Xdf = pd.DataFrame(X, columns=headers)

In [52]:
X_testdf.describe()

,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,f_goalsForHome,f_redsAgainstHome,f_redsForHome,f_shotsAgainstHome,f_shotsForHome,...,HTP,ATP,B365H,B365D,B365A,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
count,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,128.000000,1.280000e+02,1.280000e+02,1.280000e+02,...,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,128.0
mean,4.059253e-16,3.504141e-16,-9.228729e-16,-1.221245e-15,2.463307e-16,-5.204170e-17,0.000000,1.457168e-16,1.387779e-17,1.734723e-16,...,3.469447e-18,6.591949e-17,-1.179612e-16,3.400058e-16,1.387779e-17,-2.255141e-17,1.387779e-17,-1.040834e-17,-1.040834e-17,0.0
std,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929,1.003929e+00,1.003929e+00,1.003929e+00,...,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,0.0
min,-2.876803e+00,-1.572654e+00,-2.294211e+00,-1.979669e+00,-2.462765e+00,-1.484403e+00,-1.346104,-1.627226e+00,-2.686410e+00,-1.364284e+00,...,-1.814545e+00,-1.698789e+00,-7.633707e-01,-7.187656e-01,-7.777768e-01,-2.267221e+00,-2.441672e+00,-2.442455e+00,-2.598285e+00,0.0
25%,-2.915254e-01,-7.009053e-01,-8.852358e-01,-8.386421e-01,-4.532835e-01,-5.828828e-01,-0.687282,-9.743698e-01,-4.710227e-01,-6.241458e-01,...,-5.913042e-01,-5.640143e-01,-5.487847e-01,-5.473759e-01,-5.452881e-01,-6.144326e-01,-5.686638e-01,-5.573801e-01,-5.606832e-01,0.0
50%,1.952006e-01,-9.361949e-02,2.327939e-01,1.025779e-01,-2.004245e-02,-2.062275e-01,-0.270034,8.755791e-03,2.546434e-01,-2.600768e-01,...,2.201666e-02,-2.103877e-02,-2.922797e-01,-3.902687e-01,-3.515475e-01,-2.906995e-02,2.806816e-02,3.790672e-02,2.148882e-02,0.0
75%,6.859132e-01,5.148701e-01,7.367741e-01,5.854426e-01,6.796946e-01,3.996163e-01,0.519961,7.845450e-01,7.373478e-01,4.160942e-01,...,5.435094e-01,5.327741e-01,6.203654e-02,2.238778e-01,2.401467e-01,4.300380e-01,4.713548e-01,5.339791e-01,4.581179e-01,0.0
max,1.611019e+00,3.317503e+00,1.944179e+00,5.857462e+00,1.968712e+00,3.015484e+00,3.001665,2.819847e+00,1.568443e+00,3.011515e+00,...,2.827874e+00,2.609508e+00,4.797898e+00,4.937095e+00,4.429132e+00,2.955132e+00,2.877768e+00,2.419054e+00,2.815915e+00,0.0


In [53]:
Xdf.describe()

,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,f_goalsForHome,f_redsAgainstHome,f_redsForHome,f_shotsAgainstHome,f_shotsForHome,...,HTP,ATP,B365H,B365D,B365A,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
count,1.180000e+02,1.180000e+02,1.180000e+02,1.180000e+02,1.180000e+02,1.180000e+02,1.180000e+02,1.180000e+02,1.180000e+02,1.180000e+02,...,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,1.280000e+02,128.0
mean,-2.559158e-16,5.983914e-16,-4.986595e-16,1.091406e-15,5.268855e-17,2.182811e-16,-9.032323e-17,2.521523e-16,1.919369e-16,-4.516161e-17,...,-2.775558e-17,8.326673e-17,-1.387779e-17,-6.245005e-17,-9.194034e-17,-2.081668e-17,1.040834e-17,-1.561251e-17,-1.387779e-17,0.0
std,1.004264e+00,1.004264e+00,1.004264e+00,1.004264e+00,1.004264e+00,1.004264e+00,1.004264e+00,1.004264e+00,1.004264e+00,1.004264e+00,...,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,1.003929e+00,0.0
min,-2.703692e+00,-1.880066e+00,-2.097742e+00,-1.762740e+00,-2.811050e+00,-1.583511e+00,-1.351520e+00,-1.561604e+00,-2.590614e+00,-1.514108e+00,...,-1.746199e+00,-1.653854e+00,-7.617580e-01,-7.111791e-01,-7.747397e-01,-2.263941e+00,-2.454305e+00,-2.435878e+00,-2.589300e+00,0.0
25%,-5.214479e-01,-5.678237e-01,-7.860244e-01,-9.534244e-01,-4.988796e-01,-6.606755e-01,-7.572809e-01,-8.621946e-01,-4.025570e-01,-6.145519e-01,...,-5.918029e-01,-5.894526e-01,-5.189646e-01,-5.396276e-01,-5.331496e-01,-6.076481e-01,-5.441407e-01,-5.642534e-01,-5.560750e-01,0.0
50%,1.869372e-01,-7.354033e-02,-5.058669e-02,3.406931e-01,1.179681e-01,-3.639958e-01,-9.982489e-02,-1.010463e-01,2.213909e-01,-2.420183e-01,...,-1.460489e-02,-5.725208e-02,-2.901593e-01,-3.823721e-01,-3.462989e-01,-2.104422e-02,2.031609e-02,4.075199e-02,2.484650e-02,0.0
75%,5.820257e-01,3.911115e-01,8.791974e-01,6.971814e-01,7.229534e-01,7.096444e-01,4.112031e-01,8.280316e-01,6.269609e-01,3.205628e-01,...,5.625931e-01,5.414735e-01,4.955158e-02,2.323539e-01,9.056339e-02,4.124941e-01,4.644788e-01,5.360781e-01,4.605376e-01,0.0
max,2.588489e+00,3.129622e+00,1.719248e+00,2.062294e+00,1.904387e+00,2.718333e+00,2.831743e+00,2.431001e+00,2.176648e+00,2.940350e+00,...,2.799235e+00,2.603750e+00,4.805504e+00,4.950019e+00,4.459186e+00,2.969485e+00,2.875648e+00,2.418317e+00,2.813270e+00,0.0


In [54]:
test_df = pd.read_csv('data/all_seasons_joined.csv')

/home/stephen/Projects/betting_model/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (80) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
test_df = test_df.loc[test_df['season'] == 1920]

In [56]:
len(test_df)

130

In [57]:
test_df[30:31]

,Unnamed: 0,Unnamed: 0.1,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
4591,4591,30,E0,31/08/2019,Southampton,Man United,1.0,1.0,D,0.0,...,1.91,0.5,1.94,1.99,1.94,1.99,1.95,2.0,1.92,1.97
